### Setup

In [ ]:
import sys
from pathlib import Path
import os
import importlib

os.environ["MX_DEX_ENV"] = "mainnet"

sys.path.append(str(Path.cwd().parent.parent.parent.absolute()))
import config
importlib.reload(config)
from context import Context
from utils.utils_chain import WrapperAddress
from contracts.fees_collector_contract import FeesCollectorContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract

context = Context()

wasm_path = "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.4.1-rc5/fees-collector.wasm"
contract_code_hash = "4a7f6baf4aeebd1c9892b6bafd74ee3548b04be17d23fc0d554318015568d0a9"

swap_bot = "erd1p536zp9ndvlacsxkk6tv50uf9ql85vfhdkl9z7vv9xuw6q7rfrfsxqq72p"
fees_collector_contract: FeesCollectorContract = context.get_contracts(config.FEES_COLLECTORS)[0]
energy_contract: SimpleLockEnergyContract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

## Upgrade time

Upgrade and setup

In [ ]:
from time import sleep
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = fees_collector_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, wasm_path, 
                                        [], 
                                        no_init=True)
context.network_provider.check_complex_tx_status(tx_hash)
code_hash = context.network_provider.proxy.get_account(WrapperAddress(fees_collector_contract.address)).contract_code_hash.hex()
assert code_hash == contract_code_hash

In [ ]:
fees_collector_contract.set_router_address(context.deployer_account, context.network_provider.proxy, context.get_contracts(config.ROUTER_V2)[0].address)

Whitelist the swapping bot

In [ ]:
fees_collector_contract.add_admin(context.deployer_account, context.network_provider.proxy, [swap_bot])

Date reminder for next configs

In [ ]:
from datetime import datetime, timedelta
def get_next_week_start_epoch() -> int:
    first_week = fees_collector_contract.get_first_week_start_epoch(context.network_provider.proxy)
    current_week = fees_collector_contract.get_current_week(context.network_provider.proxy)
    next_week_at_epoch = first_week + current_week * 7
    return next_week_at_epoch

# Calculate date for 5th week after upgrade
current_epoch = context.network_provider.proxy.get_network_status().current_epoch
rounds_per_epoch = context.network_provider.proxy.get_network_status().raw.get("erd_rounds_per_epoch", 0)
round_at_epoch_start = context.network_provider.proxy.get_network_status().raw.get("erd_round_at_epoch_start", 0)
next_epoch_in = rounds_per_epoch - (context.network_provider.proxy.get_network_status().current_round - round_at_epoch_start)
time_at_next_epoch = timedelta(seconds=next_epoch_in * context.network_provider.proxy.get_network_config().round_duration // 1000)

first_week_start = get_next_week_start_epoch()
fifth_week_start = first_week_start + 7 * 4
epochs_until_fifth_week = fifth_week_start - current_epoch - 1
fifth_week_date = datetime.now() + timedelta(days=epochs_until_fifth_week) + time_at_next_epoch

print(f"Next configs will be applied in fifth week after upgrade, starting at epoch {fifth_week_start} around {fifth_week_date.strftime('%Y-%m-%d %H:%M')}")

## 5th week after upgrade

Cleanup known tokens list

In [ ]:
tokens = fees_collector_contract.get_reward_tokens(context.network_provider.proxy)
tokens.remove(energy_contract.base_token)
tokens.remove(energy_contract.locked_token)
context.deployer_account.sync_nonce(context.network_provider.proxy)
fees_collector_contract.remove_reward_tokens(context.deployer_account, context.network_provider.proxy, tokens)

Set burn role for MEX

In [ ]:
from contracts.builtin_contracts import ESDTContract
esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
esdt_contract.set_special_role_token(context.deployer_account, context.network_provider.proxy,
                                            [energy_contract.base_token, fees_collector_contract.address, "ESDTRoleLocalBurn"])

Modify fees for known contracts that were depositing fees

In [ ]:
from contracts.pair_contract import PairContract
known_contracts = fees_collector_contract.get_known_contracts(context.network_provider.proxy)
print(f"Retrieved {len(known_contracts)} known contracts.")

known_pairs = []
for address in known_contracts:
    mod_contract = PairContract.load_contract_by_address(address)
    if mod_contract is None:
        continue
    mod_contract.add_fees_collector(context.deployer_account, context.network_provider.proxy, [fees_collector_contract.address, 100000])
    known_pairs.append(mod_contract.address)

Enable burn on fees collector

In [ ]:
fees_collector_contract.set_base_token_burn_percent(context.deployer_account, context.network_provider.proxy, 5000)

Cleanup the known contracts list

In [ ]:
print(f"Removing {len(known_pairs)} known contracts")
fees_collector_contract.remove_known_contracts(context.deployer_account, context.network_provider.proxy, known_pairs)